Using HoloClean to clean the hospital data from the bottom section of the fairness_and_repair_exploratory_analysis notebook.

Based on https://github.com/HoloClean/holoclean/blob/master/examples/holoclean_repair_example.py .

In [1]:
import sys
sys.path.append('holoclean/')
import holoclean

In [2]:
from detect import NullDetector, ViolationDetector
from repair.featurize import *

# 1. Setup a HoloClean session.
hc = holoclean.HoloClean(
    db_name='holo',
    domain_thresh_1=0,
    domain_thresh_2=0,
    weak_label_thresh=0.99,
    max_domain=10000,
    cor_strength=0.6,
    nb_cor_strength=0.8,
    epochs=10,
    weight_decay=0.01,
    learning_rate=0.001,
    threads=1,
    batch_size=1,
    verbose=True,
    timeout=3*60000,
    feature_norm=False,
    weight_norm=False,
    print_fw=True
).session

# 2. Load training data and denial constraints.
hc.load_data('hospital', 'hospital.csv')
hc.load_dcs('holoclean/testdata/hospital_constraints.txt')
hc.ds.set_constraints(hc.get_dcs())

# 3. Detect erroneous cells using these two detectors.
detectors = [NullDetector(), ViolationDetector()]
hc.detect_errors(detectors)

# 4. Repair errors utilizing the defined features.
hc.setup_domain()
featurizers = [
    InitAttrFeaturizer(),
    OccurAttrFeaturizer(),
    FreqFeaturizer(),
    ConstraintFeaturizer(),
]

hc.repair_errors(featurizers)

16:21:57 - [DEBUG] - initiating session with parameters: {'db_user': 'holocleanuser', 'db_pwd': 'abcd1234', 'db_host': 'localhost', 'db_name': 'holo', 'threads': 1, 'timeout': 180000, 'seed': 45, 'learning_rate': 0.001, 'optimizer': 'adam', 'epochs': 10, 'weight_decay': 0.01, 'momentum': 0.0, 'batch_size': 1, 'weak_label_thresh': 0.99, 'domain_thresh_1': 0, 'domain_thresh_2': 0, 'max_domain': 10000, 'cor_strength': 0.6, 'nb_cor_strength': 0.8, 'feature_norm': False, 'weight_norm': False, 'estimator_epochs': 3, 'estimator_batch_size': 32, 'verbose': True, 'bias': False, 'print_fw': True, 'debug_mode': False}
16:21:57 - [WARNING] - Dropping the following null column from the dataset: 'Address2'
16:21:57 - [WARNING] - Dropping the following null column from the dataset: 'Address3'
16:21:57 - [ INFO] - Loaded 1000 rows with 18000 cells
16:21:58 - [DEBUG] - Time to create index: 0.00 secs
16:21:58 - [DEBUG] - Time to create index: 0.00 secs
16:21:58 - [DEBUG] - Time to create index: 0.00 se

16:21:58 - [DEBUG] - DONE extracting tuples from constraint: t1&t2&EQ(t1.MeasureName,t2.MeasureName)&IQ(t1.MeasureCode,t2.MeasureCode)
16:21:58 - [DEBUG] - DONE parsing predicate: EQ(t1.MeasureName,t2.MeasureName)
16:21:58 - [DEBUG] - DONE parsing predicate: IQ(t1.MeasureCode,t2.MeasureCode)
16:21:58 - [ INFO] - DONE Loading DCs from hospital_constraints.txt
16:21:58 - [DEBUG] - Time to load dirty data: 0.06 secs
16:21:58 - [DEBUG] - DONE with Error Detector: NullDetector in 0.06 secs
16:21:58 - [DEBUG] - Preparing to execute 15 queries.
16:21:58 - [DEBUG] - Starting to execute query SELECT t1._tid_ FROM "hospital" as t1 WHERE   EXISTS (SELECT t2._tid_ FROM "hospital" as t2 WHERE t1."Condition"=t2."Condition" AND t1."MeasureName"=t2."MeasureName" AND t1."HospitalType"<>t2."HospitalType") with id 0
16:21:58 - [DEBUG] - Time to execute query with id 0: 0.10 secs
16:21:58 - [DEBUG] - Starting to execute query SELECT t1._tid_ FROM "hospital" as t1 WHERE   EXISTS (SELECT t2._tid_ FROM "hosp

16:22:42 - [DEBUG] - Time to execute query with id 16: 0.05 secs
16:22:42 - [DEBUG] - Starting to execute query SELECT _vid_, val_id, count(*) violations FROM   "hospital" as t1, "hospital" as t2, pos_values as t3 WHERE  t1._tid_ != t2._tid_   AND  t1._tid_ = t3._tid_   AND  t3.attribute = 'HospitalName'   AND  t1."HospitalOwner"<>t2."HospitalOwner"   AND  t3.rv_val = t2."HospitalName" GROUP BY _vid_, val_id with id 17
16:22:42 - [DEBUG] - Time to execute query with id 17: 0.04 secs
16:22:42 - [DEBUG] - Starting to execute query SELECT _vid_, val_id, count(*) violations FROM   "hospital" as t1, "hospital" as t2, pos_values as t3 WHERE  t1._tid_ != t2._tid_   AND  t1._tid_ = t3._tid_   AND  t3.attribute = 'HospitalOwner'   AND  t1."HospitalName"=t2."HospitalName"   AND  t3.rv_val <> t2."HospitalOwner" GROUP BY _vid_, val_id with id 18
16:22:42 - [DEBUG] - Time to execute query with id 18: 0.04 secs
16:22:42 - [DEBUG] - Starting to execute query SELECT _vid_, val_id, count(*) violations 

100%|██████████| 10/10 [00:31<00:00,  3.09s/it]
16:23:20 - [ INFO] - DONE training repair model.
16:23:20 - [DEBUG] - Time to fit repair model: 31.35 secs
16:23:20 - [ INFO] - inferring on 11038 examples (cells)
16:23:21 - [DEBUG] - Time to execute query: 0.20 secs
16:23:23 - [DEBUG] - Time to create index: 0.00 secs
16:23:25 - [DEBUG] - Time to create index: 0.00 secs
16:23:25 - [ INFO] - DONE inferring repairs.
16:23:25 - [DEBUG] - Time to infer correct cell values: 3.11 secs
16:23:25 - [DEBUG] - Time to create table: 0.00 secs
16:23:25 - [DEBUG] - Time to create index: 0.00 secs
16:23:25 - [DEBUG] - Time to create index: 0.00 secs
16:23:25 - [ INFO] - DONE collecting the inferred values.
16:23:25 - [DEBUG] - Time to collect inferred values: 0.15 secs
16:23:25 - [ INFO] - DONE generating repaired dataset
16:23:25 - [DEBUG] - Time to store repaired dataset: 0.15 secs
16:23:25 - [ INFO] - featurizer InitAttrFeaturizer,size 17,max 1.0000,min 1.0000,avg 1.0000,abs_avg 1.0000,weights:
Pro

16:23:25 - [DEBUG] - Time to store featurizer weights: 0.00 secs


'featurizer InitAttrFeaturizer,size 17,max 1.0000,min 1.0000,avg 1.0000,abs_avg 1.0000,weights:\nProviderNumber 1.0\nHospitalName 1.0\nAddress1 1.0\nCity 1.0\nState 1.0\nZipCode 1.0\nCountyName 1.0\nPhoneNumber 1.0\nHospitalType 1.0\nHospitalOwner 1.0\nEmergencyService 1.0\nCondition 1.0\nMeasureCode 1.0\nMeasureName 1.0\nScore 1.0\nSample 1.0\nStateavg 1.0\nfeaturizer OccurAttrFeaturizer,size 289,max 0.6556,min -0.2054,avg 0.1489,abs_avg 0.1574,weights:\nProviderNumber X ProviderNumber 0.0\nProviderNumber X HospitalName 0.467\nProviderNumber X Address1 0.467\nProviderNumber X City 0.384\nProviderNumber X State 0.0\nProviderNumber X ZipCode 0.427\nProviderNumber X CountyName 0.333\nProviderNumber X PhoneNumber 0.467\nProviderNumber X HospitalType 0.002\nProviderNumber X HospitalOwner 0.061\nProviderNumber X EmergencyService 0.004\nProviderNumber X Condition -0.001\nProviderNumber X MeasureCode -0.003\nProviderNumber X MeasureName -0.002\nProviderNumber X Score -0.002\nProviderNumber X 

In [3]:
# 5. Evaluate the correctness of the results.
hc.evaluate(fpath='holoclean/testdata/hospital_clean.csv',
            tid_col='tid',
            attr_col='attribute',
            val_col='correct_val')

15:23:06 - [DEBUG] - Time to create index: 0.00 secs
15:23:06 - [DEBUG] - Time to create index: 0.00 secs
15:23:06 - [ INFO] - DONE Loading hospital_clean.csv
15:23:06 - [DEBUG] - Time to evaluate repairs: 0.80 secs
15:23:06 - [DEBUG] - Time to execute query: 0.00 secs
15:23:06 - [DEBUG] - Time to execute query: 0.00 secs
15:23:06 - [DEBUG] - Preparing to execute 17 queries.
15:23:06 - [DEBUG] - Starting to execute query SELECT count(*) FROM  "hospital" as t1, "hospital_clean" as t2 WHERE t1._tid_ = t2._tid_   AND t2._attribute_ = 'ProviderNumber'   AND t1."ProviderNumber" != t2._value_ with id 0
15:23:06 - [DEBUG] - Time to execute query with id 0: 0.04 secs
15:23:06 - [DEBUG] - Starting to execute query SELECT count(*) FROM  "hospital" as t1, "hospital_clean" as t2 WHERE t1._tid_ = t2._tid_   AND t2._attribute_ = 'HospitalName'   AND t1."HospitalName" != t2._value_ with id 1
15:23:06 - [DEBUG] - Time to execute query with id 1: 0.04 secs
15:23:06 - [DEBUG] - Starting to execute query

15:23:07 - [DEBUG] - Time to execute query with id 5: 0.04 secs
15:23:07 - [DEBUG] - Starting to execute query SELECT COUNT(*) FROM   (SELECT t2._tid_, t2._attribute_, t2._value_      FROM "hospital" as t1, "hospital_clean" as t2     WHERE t1._tid_ = t2._tid_       AND t2._attribute_ = 'CountyName'       AND t1."CountyName" != t2._value_ ) as errors, inf_values_dom as repairs WHERE errors._tid_ = repairs._tid_   AND errors._attribute_ = repairs.attribute   AND errors._value_ = repairs.rv_value with id 6
15:23:07 - [DEBUG] - Time to execute query with id 6: 0.04 secs
15:23:07 - [DEBUG] - Starting to execute query SELECT COUNT(*) FROM   (SELECT t2._tid_, t2._attribute_, t2._value_      FROM "hospital" as t1, "hospital_clean" as t2     WHERE t1._tid_ = t2._tid_       AND t2._attribute_ = 'PhoneNumber'       AND t1."PhoneNumber" != t2._value_ ) as errors, inf_values_dom as repairs WHERE errors._tid_ = repairs._tid_   AND errors._attribute_ = repairs.attribute   AND errors._value_ = repairs

15:23:08 - [ INFO] - Precision = 1.00, Recall = 0.46, Repairing Recall = 0.53, F1 = 0.63, Repairing F1 = 0.70, Detected Errors = 435, Total Errors = 509, Correct Repairs = 232, Total Repairs = 232, Total Repairs on correct cells (Grdth present) = 0, Total Repairs on incorrect cells (Grdth present) = 232
15:23:08 - [DEBUG] - Time to generate report: 1.49 secs


EvalReport(precision=1.0, recall=0.45579567779960706, repair_recall=0.5333333333333333, f1=0.6261808367071525, repair_f1=0.6956521739130436, detected_errors=435.0, total_errors=509.0, correct_repairs=232.0, total_repairs=232.0, total_repairs_grdt=232.0, total_repairs_grdt_correct=0, total_repairs_grdt_incorrect=232.0)